# ETL Process: Modelo y Real Transpuesto por GHI, POA y Energía

Este notebook recrea el proceso ETL de Power Query (Lenguaje M) en Python para combinar datos del modelo y datos reales, agregándolos a nivel diario y creando una tabla transpuesta final con comparación Real vs Modelo.

## Estructura del proceso:
1. **Configuración inicial**: Fechas, plantas e intervalos, rutas de archivos
2. **Carga de datos del Modelo**: Archivos CSV desde carpeta de modelo (iniciando en fila 11)
3. **Carga de datos EM**: Datos de estaciones meteorológicas (3 plantas)
4. **Carga de datos de Energía**: Datos de energía a nivel de 5 minutos
5. **Combinación y transformación**: Joins, limpieza de valores negativos y agregaciones
6. **Agregación horaria**: Promedio de sensores duales y agrupación por hora
7. **Agregación diaria**: Suma de energía y radiación, promedio de temperaturas
8. **Combinación final**: Join entre datos reales y modelo por día
9. **Transposición final**: Tabla con datos Real vs Modelo por métrica (GHI, POA, Energía) agrupados por día

## Métricas finales:
- **Ener_kWh**: Energía Real vs Modelo (kWh/día)
- **POA_Wh/m2**: Radiación Plano del Arreglo Real vs Modelo (Wh/m²/día)  
- **GHI_Wh/m2**: Radiación Global Horizontal Real vs Modelo (Wh/m²/día)

## Plantas incluidas:
- BSB 500, BSB 501, BSB 502, BSB 503, BSB 504

## Período de análisis:
- Desde: 2024-11-22 00:00:00
- Hasta: 2025-10-16 23:55:00
- Intervalo: 5 minutos

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 1. CONFIGURACIÓN INICIAL - Parámetros de fecha e intervalo
fecha_inicio = datetime(2024, 11, 22, 0, 0, 0)
fecha_fin = datetime(2025, 10, 16, 23, 55, 0)
intervalo_minutos = 5

# Lista de plantas
lista_plantas = ["BSB 500", "BSB 501", "BSB 502", "BSB 503", "BSB 504"]

# Rutas de archivos
ruta_base = r"C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares"
ruta_modelo = os.path.join(ruta_base, "05 IN", "04. MODELO")
ruta_energia = os.path.join(ruta_base, "05 IN", "03. ENERGÍA", "Energia_5_minutal")

print(f"Fecha inicio: {fecha_inicio}")
print(f"Fecha fin: {fecha_fin}")
print(f"Plantas: {lista_plantas}")
print(f"Ruta modelo: {ruta_modelo}")
print(f"Ruta energía: {ruta_energia}")



Fecha inicio: 2024-11-22 00:00:00
Fecha fin: 2025-10-16 23:55:00
Plantas: ['BSB 500', 'BSB 501', 'BSB 502', 'BSB 503', 'BSB 504']
Ruta modelo: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\04. MODELO
Ruta energía: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\03. ENERGÍA\Energia_5_minutal


In [3]:
# 2. CREAR GRILLA BASE DE FECHAS Y PLANTAS (Cross-Join)
def crear_grilla_base(fecha_inicio, fecha_fin, intervalo_minutos, plantas):
    """
    Crea la grilla base con todas las combinaciones de fechas-horas y plantas
    Equivalente al Cross-Join de Power Query
    """
    # Calcular número de intervalos
    duracion_total_minutos = (fecha_fin - fecha_inicio).total_seconds() / 60
    numero_intervalos = int(duracion_total_minutos / intervalo_minutos) + 1
    
    # Generar lista de fechas-horas
    fechas = []
    fecha_actual = fecha_inicio
    for i in range(numero_intervalos):
        fechas.append(fecha_actual)
        fecha_actual += timedelta(minutes=intervalo_minutos)
    
    # Crear DataFrame con todas las combinaciones
    grilla_base = []
    for fecha in fechas:
        for planta in plantas:
            grilla_base.append({
                'FechaHora': fecha,
                'Planta': planta
            })
    
    df_grilla = pd.DataFrame(grilla_base)
    return df_grilla

# Crear la grilla base
df_grilla_base = crear_grilla_base(fecha_inicio, fecha_fin, intervalo_minutos, lista_plantas)
print(f"Grilla base creada con {len(df_grilla_base):,} registros")
print(f"Rango de fechas: {df_grilla_base['FechaHora'].min()} a {df_grilla_base['FechaHora'].max()}")
print(f"Plantas únicas: {df_grilla_base['Planta'].nunique()}")
df_grilla_base.head()

Grilla base creada con 473,760 registros
Rango de fechas: 2024-11-22 00:00:00 a 2025-10-16 23:55:00
Plantas únicas: 5


,FechaHora,Planta
0,2024-11-22,BSB 500
1,2024-11-22,BSB 501
2,2024-11-22,BSB 502
3,2024-11-22,BSB 503
4,2024-11-22,BSB 504


In [5]:
# 3. CARGA DE DATOS DEL MODELO
def cargar_datos_modelo(ruta_modelo):
    """
    Carga y procesa los datos del modelo desde archivos CSV
    Equivalente a la consulta 'Modelo' en Power Query
    """
    archivos_modelo = glob.glob(os.path.join(ruta_modelo, "*.csv"))
    print(f"Archivos encontrados en modelo: {len(archivos_modelo)}")
    
    df_modelo_completo = []
    
    for archivo in archivos_modelo:
        try:
            # Leer archivo comenzando desde la fila 11 (índice 10)
            # Intentar con diferentes separadores y configuraciones
            df = None
            
            # Primero intentar leer solo para obtener el delimitador correcto
            with open(archivo, 'r', encoding='latin1') as f:
                lines = f.readlines()
                if len(lines) > 10:
                    header_line = lines[10].strip()
                    # Determinar separador basado en la línea de encabezado
                    if '\t' in header_line:
                        sep = '\t'
                    elif ';' in header_line:
                        sep = ';'
                    else:
                        sep = ','
            
            # Leer con el separador detectado
            df = pd.read_csv(archivo, skiprows=10, encoding='latin1', sep=sep, on_bad_lines='skip')
            
            # Extraer nombre de planta del archivo
            nombre_archivo = os.path.basename(archivo)
            if " - " in nombre_archivo:
                planta_temp = nombre_archivo.split(" - ")[1]
                if "_P" in planta_temp:
                    planta = planta_temp.split("_P")[0]
                elif "_" in planta_temp:
                    planta = planta_temp.split("_")[0]
                else:
                    planta = planta_temp.replace("-", " ")
            else:
                planta = nombre_archivo.split("_")[0] if "_" in nombre_archivo else nombre_archivo.split(".")[0]
            
            # Limpiar nombre de planta
            planta = planta.replace("-", " ").strip()
            
            # Añadir columna de planta
            df['Planta'] = planta
            df['Source.Name'] = nombre_archivo
            
            df_modelo_completo.append(df)
            print(f"Procesado: {archivo} -> Planta: {planta} -> Filas: {len(df)}")
            
        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
    
    if df_modelo_completo:
        df_final = pd.concat(df_modelo_completo, ignore_index=True)
        return df_final
    else:
        return pd.DataFrame()

# Cargar datos del modelo
df_modelo = cargar_datos_modelo(ruta_modelo)
if not df_modelo.empty:
    print(f"Datos del modelo cargados: {len(df_modelo):,} registros")
    print("Columnas disponibles:", df_modelo.columns.tolist())
    df_modelo.head()

Archivos encontrados en modelo: 5
Procesado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\04. MODELO\220609 - BSB-500_Project_VC3_HourlyRes_0.CSV -> Planta: BSB 500 -> Filas: 8761
Procesado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\04. MODELO\220609 - BSB-502_Project_VC3_HourlyRes_0.CSV -> Planta: BSB 502 -> Filas: 8761
Procesado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\04. MODELO\220622 - BSB-504_Project_VC4_HourlyRes_0.CSV -> Planta: BSB 504 -> Filas: 8761
Procesado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\04. MODELO\220629 - BSB-501_Project_VC3_HourlyRes_0.CSV -> Planta: BSB 501 -> Filas: 8761
Procesado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - I

In [48]:
# 4. PROCESAR DATOS DEL MODELO Y CREAR AGREGACIÓN DIARIA
def procesar_modelo(df_modelo):
    """
    Procesa los datos del modelo para crear agregación diaria
    PROCESO CORREGIDO:
    1. Primero agrupa por HORA: promedio para irradiancias, suma para energía
    2. Luego agrupa por DÍA: suma para todo (irradiancias ya están en Wh/m²)
    """
    if df_modelo.empty:
        return pd.DataFrame()
    
    # Filtrar filas válidas (similar al filtro en Power Query)
    df_filtrado = df_modelo[
        (df_modelo['date'].notna()) & 
        (df_modelo['date'] != "") &
        (df_modelo['date'] != "date") &
        (~df_modelo['date'].astype(str).str.contains("Geographical|Project|PVSYST|Simulation|Weather", na=False))
    ].copy()
    
    print(f"Registros después del filtrado: {len(df_filtrado):,}")
    
    # Convertir tipos de datos
    try:
        # Las columnas reales son: ['date', 'GlobInc', 'GlobEff', 'EArray', 'E_Grid', 'PR', 'Planta', 'Source.Name']
        # Mapear a nombres esperados:
        # date -> Fecha_y_hora
        # GlobInc -> POA_W/m²  
        # GlobEff -> GHI_W/m²
        # E_Grid -> ENE_kW
        
        # Reemplazar /90 por /2025 en fechas (como en Power Query)
        df_filtrado['date'] = df_filtrado['date'].astype(str).str.replace('/90', '/2025')
        df_filtrado['date'] = pd.to_datetime(df_filtrado['date'], errors='coerce')
        
        # Convertir columnas numéricas
        columnas_numericas = ['GlobInc', 'GlobEff', 'EArray', 'E_Grid', 'PR']
        for col in columnas_numericas:
            if col in df_filtrado.columns:
                df_filtrado[col] = pd.to_numeric(df_filtrado[col], errors='coerce')
    
    except Exception as e:
        print(f"Error en conversión de tipos: {e}")
        return pd.DataFrame()
    
    # Eliminar filas con fechas inválidas
    df_filtrado = df_filtrado.dropna(subset=['date'])
    
    # Extraer componentes de fecha (como en Power Query)
    df_filtrado['Año'] = df_filtrado['date'].dt.year
    df_filtrado['Mes'] = df_filtrado['date'].dt.month
    df_filtrado['Día'] = df_filtrado['date'].dt.day
    df_filtrado['Hora'] = df_filtrado['date'].dt.hour

    # PASO 1: Agrupar por HORA (PROMEDIO para irradiancias, SUMA para energía)
    print("Paso 1: Agregando por hora...")
    df_agrupado_hora = df_filtrado.groupby(['Año', 'Mes', 'Día', 'Hora', 'Planta']).agg({
        'E_Grid': 'sum',          # Suma para energía (E_Grid)
        'GlobInc': 'mean',        # PROMEDIO para POA (se convierte a Wh/m² en esta etapa)
        'GlobEff': 'mean'         # PROMEDIO para GHI (se convierte a Wh/m² en esta etapa)
    }).reset_index()
    
    print(f"Datos del modelo agrupados por hora: {len(df_agrupado_hora):,} registros")

    # PASO 2: Agrupar por DÍA (SUMA para todo, ya que irradiancias están en Wh/m²)
    print("Paso 2: Agregando por día...")
    df_agrupado_dia = df_agrupado_hora.groupby(['Año', 'Mes', 'Día', 'Planta']).agg({
        'E_Grid': 'sum',          # Suma para energía (E_Grid)
        'GlobInc': 'sum',         # SUMA para POA (ya está en Wh/m² después de agregación horaria)
        'GlobEff': 'sum'          # SUMA para GHI (ya está en Wh/m² después de agregación horaria)
    }).reset_index()
    
    # Renombrar columnas para claridad
    df_agrupado_dia = df_agrupado_dia.rename(columns={
        'E_Grid': 'Ener_kWh',
        'GlobInc': 'POA_Wh/m2',
        'GlobEff': 'GHI_Wh/m2'
    })
    
    print(f"Datos del modelo agrupados por día: {len(df_agrupado_dia):,} registros")
    return df_agrupado_dia

# Procesar datos del modelo
if not df_modelo.empty:
    df_modelo_dia = procesar_modelo(df_modelo)
    if not df_modelo_dia.empty:
        print("Primeros registros del modelo agrupado por día:")
        print(df_modelo_dia.head())
        print(f"Rango de fechas: {df_modelo_dia['Año'].min()}-{df_modelo_dia['Mes'].min()}-{df_modelo_dia['Día'].min()} a {df_modelo_dia['Año'].max()}-{df_modelo_dia['Mes'].max()}-{df_modelo_dia['Día'].max()}")
    else:
        print("Error: No se pudieron procesar los datos del modelo")
        df_modelo_dia = pd.DataFrame()
else:
    print("Error: No hay datos del modelo para procesar")
    df_modelo_dia = pd.DataFrame()

Registros después del filtrado: 43,805
Paso 1: Agregando por hora...
Datos del modelo agrupados por hora: 43,800 registros
Paso 2: Agregando por día...
Datos del modelo agrupados por día: 1,825 registros
Primeros registros del modelo agrupado por día:
    Año  Mes  Día   Planta  Ener_kWh  POA_Wh/m2  GHI_Wh/m2
0  2025    1    1  BSB 500  103874.0      925.0        0.0
1  2025    1    1  BSB 501  103762.0      925.0        0.0
2  2025    1    1  BSB 502  112715.0      925.0        0.0
3  2025    1    1  BSB 503  103028.0      925.0      210.0
4  2025    1    1  BSB 504  102719.0      925.0        0.0
Rango de fechas: 2025-1-1 a 2025-12-31
Paso 1: Agregando por hora...
Datos del modelo agrupados por hora: 43,800 registros
Paso 2: Agregando por día...
Datos del modelo agrupados por día: 1,825 registros
Primeros registros del modelo agrupado por día:
    Año  Mes  Día   Planta  Ener_kWh  POA_Wh/m2  GHI_Wh/m2
0  2025    1    1  BSB 500  103874.0      925.0        0.0
1  2025    1    1  BSB 5

In [34]:
# 5. CARGA Y PROCESAMIENTO DE DATOS EM (ESTACIONES METEOROLÓGICAS) 
def cargar_datos_em():
    """
    Carga y procesa datos de estaciones meteorológicas
    Equivalente a la consulta 'EM' en Power Query
    """
    # Rutas de archivos EM (corregida) archivos tipo TXT  
    ruta_em_base = os.path.join(ruta_base, "05 IN", "02. EEMM")  

    print(f"Buscando archivos EM en: {ruta_em_base}")
    
    # Buscar archivos de EM - versión mejorada
    archivos_em = []
    
    # Mapeo de plantas a patrones de archivos
    patrones_plantas = {
        "BSB 500": "BSB500_MeasData",
        "BSB 501": "BSB501_MeasData", 
        "BSB 502": "BSB502_MeasData"
    }
    
    for planta, patron in patrones_plantas.items():
        # Buscar archivos con diferentes extensiones (.TXT, .txt)
        for extension in ['.TXT', '.txt']:
            archivo_path = os.path.join(ruta_em_base, f"{patron}{extension}")
            if os.path.exists(archivo_path):
                archivos_em.append((archivo_path, planta))
                print(f"Encontrado: {archivo_path} -> {planta}")
                break  # Solo tomar el primer archivo encontrado por planta
    
    print(f"Archivos EM encontrados: {len(archivos_em)}")
    
    if not archivos_em:
        print("No se encontraron archivos EM, creando DataFrame vacío")
        return pd.DataFrame()
    
    df_em_completo = []
    
    for archivo, planta in archivos_em:
        try:
            print(f"Procesando: {archivo}")
            
            # Primero leer el encabezado (línea 2) para obtener nombres de columnas
            with open(archivo, 'r', encoding='latin1') as f:
                lines = f.readlines()
                if len(lines) < 5:
                    print(f"Archivo {archivo} no tiene suficientes líneas")
                    continue
                
                # La línea 2 (índice 1) contiene los nombres de columnas
                header_line = lines[1].strip().strip('"')
                column_names = [col.strip().strip('"') for col in header_line.split('","')]
                
                print(f"Columnas detectadas: {column_names[:5]}...")  # Mostrar solo las primeras 5
            
            # Leer archivo saltando las primeras 4 líneas y usando los nombres de columnas detectados
            df_em = pd.read_csv(archivo, encoding='latin1', skiprows=4, names=column_names)
            
            print(f"Filas leídas: {len(df_em)}")
            print(f"Columnas finales: {df_em.columns.tolist()}")
            
            # Verificar que existe la columna TIMESTAMP
            if 'TIMESTAMP' not in df_em.columns:
                print(f"Advertencia: No se encontró columna TIMESTAMP en {archivo}")
                continue
            
            # Filtrar filas válidas
            df_em = df_em[
                (df_em['TIMESTAMP'].notna()) & 
                (df_em['TIMESTAMP'] != "") &
                (df_em['TIMESTAMP'] != "TS")
            ].copy()
            
            print(f"Filas después del filtrado: {len(df_em)}")
            
            # Mapear nombres de columnas según Power Query
            mapeo_columnas = {}
            for col in df_em.columns:
                if 'Irrad_POA_1' in col:
                    mapeo_columnas[col] = 'POA_1_W/m2'
                elif 'Irrad_GHI_1' in col:
                    mapeo_columnas[col] = 'GHI_1_W/m2'
                elif 'Irrad_POA_2' in col:
                    mapeo_columnas[col] = 'POA_2_W/m2'
                elif 'Irrad_GHI_2' in col:
                    mapeo_columnas[col] = 'GHI_2_W/m2'
                elif 'Temp_Modulo_1' in col:
                    mapeo_columnas[col] = 'Tmod_1_C'
                elif 'Temp_Modulo_2' in col:
                    mapeo_columnas[col] = 'Tmod_2_C'
                elif 'Temp_Amb_1' in col:
                    mapeo_columnas[col] = 'Tamb_1_C'
                elif 'Temp_Amb_2' in col:
                    mapeo_columnas[col] = 'Tamb_2_C'
                elif 'Voltaje' in col:
                    mapeo_columnas[col] = 'Voltaje_V'
                elif 'Irrad_Reflejada_1' in col:
                    mapeo_columnas[col] = 'Albedo_1_W/m2'
                elif 'HumRel' in col:
                    mapeo_columnas[col] = 'Humedad_%'
                elif 'Soiling' in col:
                    mapeo_columnas[col] = 'Soiling'
            
            # Renombrar columnas
            df_em = df_em.rename(columns=mapeo_columnas)
            print(f"Columnas después del mapeo: {[col for col in df_em.columns if col in mapeo_columnas.values() or col == 'TIMESTAMP']}")
            
            # Agregar columna de planta
            df_em['Planta'] = planta
            
            # Eliminar columnas no necesarias (como en Power Query)
            columnas_a_eliminar = [
                'RECORD', 'Vel_Viento_m/s', 'Dir_Viento_Deg', 'Dir_Viento_Std_Deg', 
                'PAtm_hPa', 'usri_1', 'SbLarga_500_Precip_5m', 'ustr1_raw',
                'SbLarga_500_Vel_Viento_5m_Avg', 'SbLarga_500_Dir_Viento_5m_Avg',
                'SbLarga_500_Dir_Viento_5m_Std', 'SbLarga_500_PAtm_5m_Avg'
            ]
            df_em = df_em.drop(columns=[col for col in columnas_a_eliminar if col in df_em.columns])
            
            df_em_completo.append(df_em)
            print(f"✅ Procesado EM: {archivo} -> Planta: {planta} -> Filas: {len(df_em)}")
            
        except Exception as e:
            print(f"❌ Error procesando EM {archivo}: {e}")
            import traceback
            print(traceback.format_exc())
    
    if df_em_completo:
        df_em_final = pd.concat(df_em_completo, ignore_index=True)
        
        print(f"Total de registros antes del procesamiento: {len(df_em_final)}")
        
        # Procesar datos según Power Query
        # Reemplazar puntos por comas y "NAN" por valores nulos
        columnas_numericas = [
            'POA_1_W/m2', 'GHI_1_W/m2', 'POA_2_W/m2', 'GHI_2_W/m2', 
            'Tmod_1_C', 'Tmod_2_C', 'Tamb_1_C', 'Tamb_2_C', 
            'Voltaje_V', 'Albedo_1_W/m2', 'Humedad_%', 'Soiling'
        ]
        
        for col in columnas_numericas:
            if col in df_em_final.columns:
                # Convertir a string primero para hacer reemplazos
                df_em_final[col] = df_em_final[col].astype(str)
                # Reemplazar patrones comunes de datos faltantes
                df_em_final[col] = df_em_final[col].str.replace('NAN', '', regex=False)
                df_em_final[col] = df_em_final[col].str.replace('nan', '', regex=False)
                df_em_final[col] = df_em_final[col].str.replace('NULL', '', regex=False)
                # Convertir a numérico
                df_em_final[col] = pd.to_numeric(df_em_final[col], errors='coerce')
        
        # Convertir timestamp
        df_em_final['TIMESTAMP'] = pd.to_datetime(df_em_final['TIMESTAMP'], errors='coerce')
        df_em_final = df_em_final.dropna(subset=['TIMESTAMP'])
        
        print(f"Registros después de limpieza de timestamp: {len(df_em_final)}")
        
        # Crear copia del timestamp como texto (como en Power Query)
        df_em_final['TIMESTAMP - Copia'] = df_em_final['TIMESTAMP'].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        return df_em_final
    else:
        return pd.DataFrame()

# Cargar datos EM
print("=== CARGANDO DATOS DE ESTACIONES METEOROLÓGICAS (CORREGIDO V2) ===")
df_em = cargar_datos_em()
if not df_em.empty:
    print(f"\n✅ Datos EM cargados exitosamente: {len(df_em):,} registros")
    print("Columnas disponibles:", df_em.columns.tolist())
    print(f"Rango de fechas: {df_em['TIMESTAMP'].min()} a {df_em['TIMESTAMP'].max()}")
    print("Plantas en EM:", df_em['Planta'].unique())
    print("\nPrimeras filas:")
    print(df_em.head())
    
    # Mostrar estadísticas por planta
    print("\n=== ESTADÍSTICAS POR PLANTA ===")
    for planta in df_em['Planta'].unique():
        df_planta = df_em[df_em['Planta'] == planta]
        print(f"\n{planta}:")
        print(f"  - Registros: {len(df_planta):,}")
        print(f"  - Rango fechas: {df_planta['TIMESTAMP'].min()} a {df_planta['TIMESTAMP'].max()}")
        # Mostrar algunas métricas clave
        for col in ['POA_1_W/m2', 'GHI_1_W/m2', 'Tmod_1_C', 'Tamb_1_C']:
            if col in df_planta.columns:
                valores_validos = df_planta[col].dropna()
                if len(valores_validos) > 0:
                    print(f"  - {col}: {valores_validos.mean():.2f} ± {valores_validos.std():.2f}")
else:
    print("❌ Advertencia: No se pudieron cargar datos EM")

=== CARGANDO DATOS DE ESTACIONES METEOROLÓGICAS (CORREGIDO V2) ===
Buscando archivos EM en: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\02. EEMM
Encontrado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\02. EEMM\BSB500_MeasData.TXT -> BSB 500
Encontrado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\02. EEMM\BSB501_MeasData.TXT -> BSB 501
Encontrado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\02. EEMM\BSB502_MeasData.TXT -> BSB 502
Archivos EM encontrados: 3
Procesando: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\02. EEMM\BSB500_MeasData.TXT
Columnas detectadas: ['TIMESTAMP', 'RECORD', 'SbLarga_500_Irrad_POA_1_5m_Avg', 'SbLarga_500_Irr

In [35]:
df_em.head()

,TIMESTAMP,POA_1_W/m2,GHI_1_W/m2,POA_2_W/m2,GHI_2_W/m2,Albedo_1_W/m2,Humedad_%,Tmod_1_C,Tmod_2_C,Soiling,Tamb_1_C,Tamb_2_C,Voltaje_V,Planta,TIMESTAMP - Copia
0,2025-02-14 16:55:00,56.76832,50.93786,56.67333,54.65786,13.81240,61.00000,22.92174,22.86534,NaN,24.79260,25.11460,12.35401,BSB 500,2025-02-14 16:55:00
1,2025-02-14 17:00:00,71.15778,63.52339,71.07851,68.73905,13.79694,61.00000,29.07562,29.00211,NaN,28.93294,29.30270,12.35115,BSB 500,2025-02-14 17:00:00
2,2025-02-14 17:05:00,67.68255,60.23819,67.64599,65.92947,12.80262,61.15278,28.93820,28.86582,1.035138,28.87198,29.30294,12.34984,BSB 500,2025-02-14 17:05:00
3,2025-02-14 17:10:00,44.40710,39.74600,44.49584,43.66600,12.38411,62.00000,21.10587,21.05112,1.033209,23.04978,23.43686,12.35538,BSB 500,2025-02-14 17:10:00
4,2025-02-14 17:15:00,54.27269,48.76986,54.39705,53.57546,11.65897,62.00000,27.79878,27.72180,1.035332,28.00419,28.42916,12.35045,BSB 500,2025-02-14 17:15:00


In [36]:
df_em.columns

Index(['TIMESTAMP', 'POA_1_W/m2', 'GHI_1_W/m2', 'POA_2_W/m2', 'GHI_2_W/m2',
       'Albedo_1_W/m2', 'Humedad_%', 'Tmod_1_C', 'Tmod_2_C', 'Soiling',
       'Tamb_1_C', 'Tamb_2_C', 'Voltaje_V', 'Planta', 'TIMESTAMP - Copia'],
      dtype='object')

In [37]:
# VERIFICACIÓN RÁPIDA DE CARGA DE DATOS EM
print("=== VERIFICACIÓN DE DATOS EM ===")
print(f"Tipo de df_em: {type(df_em)}")
print(f"Forma de df_em: {df_em.shape if hasattr(df_em, 'shape') else 'No disponible'}")
if not df_em.empty:
    print(f"Columnas: {df_em.columns.tolist()}")
    print(f"Plantas únicas: {df_em['Planta'].unique()}")
    print(f"Rango de fechas: {df_em['TIMESTAMP'].min()} a {df_em['TIMESTAMP'].max()}")
    print("\nMuestra de datos:")
    print(df_em[['TIMESTAMP', 'Planta', 'POA_1_W/m2', 'GHI_1_W/m2', 'Tmod_1_C']].head())
else:
    print("DataFrame vacío")

=== VERIFICACIÓN DE DATOS EM ===
Tipo de df_em: <class 'pandas.core.frame.DataFrame'>
Forma de df_em: (170497, 15)
Columnas: ['TIMESTAMP', 'POA_1_W/m2', 'GHI_1_W/m2', 'POA_2_W/m2', 'GHI_2_W/m2', 'Albedo_1_W/m2', 'Humedad_%', 'Tmod_1_C', 'Tmod_2_C', 'Soiling', 'Tamb_1_C', 'Tamb_2_C', 'Voltaje_V', 'Planta', 'TIMESTAMP - Copia']
Plantas únicas: ['BSB 500' 'BSB 501' 'BSB 502']
Rango de fechas: 2024-11-22 15:05:00 a 2025-10-15 14:55:00

Muestra de datos:
            TIMESTAMP   Planta  POA_1_W/m2  GHI_1_W/m2  Tmod_1_C
0 2025-02-14 16:55:00  BSB 500    56.76832    50.93786  22.92174
1 2025-02-14 17:00:00  BSB 500    71.15778    63.52339  29.07562
2 2025-02-14 17:05:00  BSB 500    67.68255    60.23819  28.93820
3 2025-02-14 17:10:00  BSB 500    44.40710    39.74600  21.10587
4 2025-02-14 17:15:00  BSB 500    54.27269    48.76986  27.79878


In [39]:
# 6. CARGA Y PROCESAMIENTO DE DATOS DE ENERGÍA
def cargar_datos_energia(ruta_energia):
    """
    Carga y procesa datos de energía de 5 minutos
    Equivalente a la consulta 'Energia' en Power Query
    """
    # Buscar archivos Excel (.xlsx) en lugar de CSV
    archivos_energia = glob.glob(os.path.join(ruta_energia, "*.xlsx"))
    print(f"Archivos de energía encontrados: {len(archivos_energia)}")
    
    # Mostrar archivos encontrados para debugging
    for archivo in archivos_energia:
        print(f"  - {os.path.basename(archivo)}")
    
    # C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\03. ENERGÍA\Energia_5_minutal
    # esta es la ruta correcta, valida, son horas de calculo de microsoft Excel
    if not archivos_energia:
        print("No se encontraron archivos Excel en la ruta especificada")
        return pd.DataFrame()
    
    df_energia_completo = []
    
    for archivo in archivos_energia:
        try:
            # Leer archivo Excel - primera hoja por defecto
            print(f"Leyendo archivo: {archivo}")
            df = pd.read_excel(archivo, engine='openpyxl')
            
            print(f"Columnas encontradas: {df.columns.tolist()}")
            print(f"Primeras filas del archivo {os.path.basename(archivo)}:")
            print(df.head(3))
            
            # Extraer nombre de planta del archivo (como en Power Query)
            nombre_archivo = os.path.basename(archivo)
            # Extraer después de "BSB " y antes de "_5"
            if "BSB " in nombre_archivo:
                # Extraer "BSB 500", "BSB 501", etc.
                planta_part = nombre_archivo.split("BSB ")[1]
                if "_5" in planta_part:
                    planta = "BSB " + planta_part.split("_5")[0]
                else:
                    planta = "BSB " + planta_part.split(".")[0]
            elif "a_" in nombre_archivo:
                # Fallback para el patrón original
                planta = nombre_archivo.split("a_")[1].split("_5")[0] if "_5" in nombre_archivo else nombre_archivo.split("a_")[1].split(".")[0]
            else:
                planta = nombre_archivo.split("_")[0]
            
            df['Planta'] = planta
            df['Source.Name'] = nombre_archivo
            
            df_energia_completo.append(df)
            print(f"✅ Procesado energía: {archivo} -> Planta: {planta} -> Filas: {len(df)}")
            
        except Exception as e:
            print(f"❌ Error procesando energía {archivo}: {e}")
            import traceback
            print(traceback.format_exc())
    
    if df_energia_completo:
        df_final = pd.concat(df_energia_completo, ignore_index=True)
        
        print(f"DataFrame combinado - Forma: {df_final.shape}")
        print(f"Columnas disponibles: {df_final.columns.tolist()}")
        
        # Verificar qué columnas de tiempo están disponibles
        columnas_tiempo = [col for col in df_final.columns if any(keyword in col.lower() for keyword in ['hora', 'time', 'fecha', 'date', 'timestamp'])]
        print(f"Columnas de tiempo detectadas: {columnas_tiempo}")
        
        # Buscar la columna de tiempo correcta
        columna_tiempo = None
        if 'Hora' in df_final.columns:
            columna_tiempo = 'Hora'
        elif 'HORA' in df_final.columns:
            columna_tiempo = 'HORA'
        elif 'Fecha y Hora' in df_final.columns:
            columna_tiempo = 'Fecha y Hora'
        elif 'FECHA Y HORA' in df_final.columns:
            columna_tiempo = 'FECHA Y HORA'
        elif len(columnas_tiempo) > 0:
            columna_tiempo = columnas_tiempo[0]
        
        if columna_tiempo:
            print(f"Usando columna de tiempo: {columna_tiempo}")
            # Convertir tipos
            df_final[columna_tiempo] = pd.to_datetime(df_final[columna_tiempo], errors='coerce')
            
            # Renombrar a 'Hora' para consistencia
            if columna_tiempo != 'Hora':
                df_final = df_final.rename(columns={columna_tiempo: 'Hora'})
        else:
            print("❌ No se encontró columna de tiempo válida")
            return pd.DataFrame()
        
        # Buscar la columna de energía
        columnas_energia = [col for col in df_final.columns if any(keyword in col.lower() for keyword in ['energía', 'energia', 'kwh', 'energy'])]
        print(f"Columnas de energía detectadas: {columnas_energia}")
        
        columna_energia = None
        if 'ENERGÍA ACTIVA (kWh)' in df_final.columns:
            columna_energia = 'ENERGÍA ACTIVA (kWh)'
        elif 'Energía Activa (kWh)' in df_final.columns:
            columna_energia = 'Energía Activa (kWh)'
        elif 'ENERGIA ACTIVA (kWh)' in df_final.columns:
            columna_energia = 'ENERGIA ACTIVA (kWh)'
        elif len(columnas_energia) > 0:
            columna_energia = columnas_energia[0]
        
        if columna_energia:
            print(f"Usando columna de energía: {columna_energia}")
            df_final[columna_energia] = pd.to_numeric(df_final[columna_energia], errors='coerce')
            
            # Renombrar a nombre estándar
            if columna_energia != 'ENERGÍA ACTIVA (kWh)':
                df_final = df_final.rename(columns={columna_energia: 'ENERGÍA ACTIVA (kWh)'})
        else:
            print("❌ No se encontró columna de energía válida")
            return pd.DataFrame()
        
        # Eliminar filas con fechas inválidas
        df_final = df_final.dropna(subset=['Hora'])
        print(f"Registros después de eliminar fechas inválidas: {len(df_final)}")
        
        # Redondear hora hacia abajo a intervalos de 5 minutos (como en Power Query)
        intervalos_por_dia = 24 * (60 / 5)  # 288
        df_final['Hora'] = df_final['Hora'].apply(
            lambda x: pd.Timestamp.fromordinal(int(pd.Timestamp(x).toordinal())) + 
                     pd.Timedelta(minutes=int((pd.Timestamp(x).hour * 60 + pd.Timestamp(x).minute) // 5) * 5)
        )
        
        # Crear clave de join (como en Power Query)
        df_final['ClaveJoin'] = df_final['Planta'] + "|" + df_final['Hora'].dt.strftime('%Y-%m-%dT%H:%M')
        
        return df_final
    else:
        return pd.DataFrame()

# Cargar datos de energía
print("=== CARGANDO DATOS DE ENERGÍA (CORREGIDO PARA EXCEL) ===")
print(f"Ruta general de carga: {ruta_energia}")
df_energia = cargar_datos_energia(ruta_energia)
if not df_energia.empty:
    print(f"\n✅ Datos de energía cargados: {len(df_energia):,} registros")
    print("Columnas disponibles:", df_energia.columns.tolist())
    print(f"Rango de fechas: {df_energia['Hora'].min()} a {df_energia['Hora'].max()}")
    print("Plantas en energía:", df_energia['Planta'].unique())
    print("\nPrimeras filas:")
    print(df_energia.head())
    
    # Mostrar estadísticas por planta
    print("\n=== ESTADÍSTICAS POR PLANTA ===")
    for planta in df_energia['Planta'].unique():
        df_planta_energia = df_energia[df_energia['Planta'] == planta]
        print(f"\n{planta}:")
        print(f"  - Registros: {len(df_planta_energia):,}")
        print(f"  - Rango fechas: {df_planta_energia['Hora'].min()} a {df_planta_energia['Hora'].max()}")
        if 'ENERGÍA ACTIVA (kWh)' in df_planta_energia.columns:
            energia_valida = df_planta_energia['ENERGÍA ACTIVA (kWh)'].dropna()
            if len(energia_valida) > 0:
                print(f"  - Energía total: {energia_valida.sum():.2f} kWh")
                print(f"  - Energía promedio: {energia_valida.mean():.2f} kWh")
else:
    print("❌ Advertencia: No se pudieron cargar datos de energía")

=== CARGANDO DATOS DE ENERGÍA (CORREGIDO PARA EXCEL) ===
Ruta general de carga: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\03. ENERGÍA\Energia_5_minutal
Archivos de energía encontrados: 3
  - Energía_BSB 500_5_MINUTAL.xlsx
  - Energía_BSB 501_5_MINUTAL.xlsx
  - Energía_BSB 502_5_MINUTAL.xlsx
Leyendo archivo: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\03. ENERGÍA\Energia_5_minutal\Energía_BSB 500_5_MINUTAL.xlsx
Columnas encontradas: ['Hora', 'ENERGÍA ACTIVA (kWh)']
Primeras filas del archivo Energía_BSB 500_5_MINUTAL.xlsx:
                 Hora  ENERGÍA ACTIVA (kWh)
0 2024-11-23 00:05:00                -2.500
1 2024-11-23 00:10:00                -2.376
2 2024-11-23 00:15:00                -2.406
✅ Procesado energía: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 IN\03. ENERGÍA\E

In [40]:
# 7. COMBINACIÓN Y TRANSFORMACIÓN - JOINS Y AGREGACIONES
def procesar_datos_reales(df_grilla_base, df_em, df_energia):
    """
    Combina grilla base con datos EM y energía, procesa y agrega datos reales
    Equivalente al proceso principal de Power Query
    """
    # Paso 1: Convertir FechaHora de grilla a texto para join con EM
    df_trabajo = df_grilla_base.copy()
    df_trabajo['FechaHora_texto'] = df_trabajo['FechaHora'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Paso 2: Left Join con EM
    if not df_em.empty:
        df_con_em = df_trabajo.merge(
            df_em, 
            left_on=['Planta', 'FechaHora_texto'], 
            right_on=['Planta', 'TIMESTAMP - Copia'], 
            how='left'
        )
        print(f"Después del join con EM: {len(df_con_em):,} registros")
    else:
        print("Advertencia: No hay datos EM, continuando sin ellos")
        df_con_em = df_trabajo.copy()
        # Agregar columnas EM vacías
        columnas_em = ['POA_1_W/m2', 'GHI_1_W/m2', 'POA_2_W/m2', 'GHI_2_W/m2', 
                      'Tmod_1_C', 'Tmod_2_C', 'Tamb_1_C', 'Tamb_2_C', 
                      'Voltaje_V', 'Albedo_1_W/m2', 'Humedad_%', 'Soiling']
        for col in columnas_em:
            df_con_em[col] = np.nan
    
    # Paso 3: Redondear hora hacia abajo y crear clave para energía
    intervalos_por_dia = 24 * (60 / 5)  # 288
    
    # Verificar que columna FechaHora esté presente
    fecha_col = 'FechaHora' if 'FechaHora' in df_con_em.columns else 'FechaHora_x'
    if fecha_col not in df_con_em.columns:
        print("Columnas disponibles:", df_con_em.columns.tolist())
        fecha_col = df_con_em.columns[0]  # Usar la primera columna como respaldo
    
    df_con_em['FechaHora_redondeada'] = df_con_em[fecha_col].apply(
        lambda x: pd.Timestamp.fromordinal(int(x.toordinal())) + 
                 pd.Timedelta(minutes=int((x.hour * 60 + x.minute) // 5) * 5)
    )
    
    # Crear clave de join para energía
    df_con_em['ClaveJoin'] = df_con_em['Planta'] + "|" + df_con_em['FechaHora_redondeada'].dt.strftime('%Y-%m-%dT%H:%M')
    
    # Paso 4: Left Join con Energía
    if not df_energia.empty:
        df_completo = df_con_em.merge(
            df_energia[['ClaveJoin', 'ENERGÍA ACTIVA (kWh)']], 
            on='ClaveJoin', 
            how='left'
        )
        print(f"Después del join con Energía: {len(df_completo):,} registros")
    else:
        print("Advertencia: No hay datos de energía, continuando sin ellos")
        df_completo = df_con_em.copy()
        df_completo['ENERGÍA ACTIVA (kWh)'] = np.nan
    
    # Paso 5: Agregar columnas de fecha
    df_completo['ano'] = df_completo[fecha_col].dt.year
    df_completo['mes'] = df_completo[fecha_col].dt.month
    df_completo['dia'] = df_completo[fecha_col].dt.day
    df_completo['hora'] = df_completo[fecha_col].dt.hour
    df_completo['hora_completa'] = df_completo[fecha_col].dt.time
    
    return df_completo

# Procesar datos reales
print("Combinando datos reales...")
df_datos_reales = procesar_datos_reales(df_grilla_base, df_em, df_energia)
print(f"Datos combinados: {len(df_datos_reales):,} registros")
print("Primeros registros combinados:")

Combinando datos reales...
Después del join con EM: 502,667 registros
Después del join con EM: 502,667 registros
Después del join con Energía: 502,670 registros
Datos combinados: 502,670 registros
Primeros registros combinados:
   FechaHora   Planta      FechaHora_texto TIMESTAMP  POA_1_W/m2  GHI_1_W/m2  \
0 2024-11-22  BSB 500  2024-11-22 00:00:00       NaT         NaN         NaN   
1 2024-11-22  BSB 501  2024-11-22 00:00:00       NaT         NaN         NaN   
2 2024-11-22  BSB 502  2024-11-22 00:00:00       NaT         NaN         NaN   
3 2024-11-22  BSB 503  2024-11-22 00:00:00       NaT         NaN         NaN   
4 2024-11-22  BSB 504  2024-11-22 00:00:00       NaT         NaN         NaN   

   POA_2_W/m2  GHI_2_W/m2  Albedo_1_W/m2  Humedad_%  ...  Voltaje_V  \
0         NaN         NaN            NaN        NaN  ...        NaN   
1         NaN         NaN            NaN        NaN  ...        NaN   
2         NaN         NaN            NaN        NaN  ...        NaN   
3      

In [42]:
df_datos_reales.tail()

,FechaHora,Planta,FechaHora_texto,TIMESTAMP,POA_1_W/m2,GHI_1_W/m2,POA_2_W/m2,GHI_2_W/m2,Albedo_1_W/m2,Humedad_%,...,Voltaje_V,TIMESTAMP - Copia,FechaHora_redondeada,ClaveJoin,ENERGÍA ACTIVA (kWh),ano,mes,dia,hora,hora_completa
502665,2025-10-16 23:55:00,BSB 500,2025-10-16 23:55:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2025-10-16 23:55:00,BSB 500|2025-10-16T23:55,NaN,2025,10,16,23,23:55:00
502666,2025-10-16 23:55:00,BSB 501,2025-10-16 23:55:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2025-10-16 23:55:00,BSB 501|2025-10-16T23:55,NaN,2025,10,16,23,23:55:00
502667,2025-10-16 23:55:00,BSB 502,2025-10-16 23:55:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2025-10-16 23:55:00,BSB 502|2025-10-16T23:55,NaN,2025,10,16,23,23:55:00
502668,2025-10-16 23:55:00,BSB 503,2025-10-16 23:55:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2025-10-16 23:55:00,BSB 503|2025-10-16T23:55,NaN,2025,10,16,23,23:55:00
502669,2025-10-16 23:55:00,BSB 504,2025-10-16 23:55:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2025-10-16 23:55:00,BSB 504|2025-10-16T23:55,NaN,2025,10,16,23,23:55:00


In [43]:
# 8. LIMPIAR VALORES NEGATIVOS Y AGREGACIÓN HORARIA
def limpiar_y_agregar_por_hora(df_datos_reales):
    """
    Limpia valores negativos, promedia sensores duales y agrega por hora
    Equivalente a la limpieza y agregación horaria de Power Query
    """
    df_limpio = df_datos_reales.copy()
    
    # Reemplazar valores negativos por 0 (como en Power Query)
    columnas_a_limpiar = [
        'POA_1_W/m2', 'GHI_1_W/m2', 'POA_2_W/m2', 'GHI_2_W/m2', 
        'Tmod_1_C', 'Tmod_2_C', 'Tamb_1_C', 'Tamb_2_C', 
        'Voltaje_V', 'Albedo_1_W/m2', 'Humedad_%', 'Soiling', 
        'ENERGÍA ACTIVA (kWh)'
    ]
    
    for col in columnas_a_limpiar:
        if col in df_limpio.columns:
            df_limpio[col] = df_limpio[col].fillna(0)
            df_limpio[col] = df_limpio[col].apply(lambda x: 0 if x <= 0 else x)
    
    # Agregación por hora (como en Power Query)
    df_agrupado_hora = df_limpio.groupby(['ano', 'mes', 'dia', 'hora', 'Planta']).agg({
        'POA_1_W/m2': 'mean',
        'GHI_1_W/m2': 'mean', 
        'POA_2_W/m2': 'mean',
        'GHI_2_W/m2': 'mean',
        'Tmod_1_C': 'mean',
        'Tmod_2_C': 'mean',
        'Tamb_1_C': 'mean',
        'Tamb_2_C': 'mean',
        'ENERGÍA ACTIVA (kWh)': 'sum'
    }).reset_index()
    
    # Promediar sensores duales (como en Power Query)
    df_agrupado_hora['POA_W/m2'] = (df_agrupado_hora['POA_1_W/m2'] + df_agrupado_hora['POA_2_W/m2']) / 2
    df_agrupado_hora['GHI_W/m2'] = (df_agrupado_hora['GHI_1_W/m2'] + df_agrupado_hora['GHI_2_W/m2']) / 2
    df_agrupado_hora['Tmod_C'] = (df_agrupado_hora['Tmod_1_C'] + df_agrupado_hora['Tmod_2_C']) / 2
    df_agrupado_hora['Tamb_C'] = (df_agrupado_hora['Tamb_1_C'] + df_agrupado_hora['Tamb_2_C']) / 2
    
    # Eliminar columnas originales de sensores individuales
    columnas_a_eliminar = [
        'POA_1_W/m2', 'POA_2_W/m2', 'GHI_1_W/m2', 'GHI_2_W/m2',
        'Tmod_1_C', 'Tmod_2_C', 'Tamb_1_C', 'Tamb_2_C'
    ]
    df_agrupado_hora = df_agrupado_hora.drop(columns=columnas_a_eliminar)
    
    # Renombrar columna de energía
    df_agrupado_hora = df_agrupado_hora.rename(columns={'ENERGÍA ACTIVA (kWh)': 'Energia_kWh'})
    
    print(f"Datos agrupados por hora: {len(df_agrupado_hora):,} registros")
    return df_agrupado_hora

# Limpiar y agregar por hora
df_agrupado_hora = limpiar_y_agregar_por_hora(df_datos_reales)
print("Primeros registros agrupados por hora:")
print(df_agrupado_hora.head())

Datos agrupados por hora: 39,480 registros
Primeros registros agrupados por hora:
    ano  mes  dia  hora   Planta  Energia_kWh  POA_W/m2  GHI_W/m2  Tmod_C  \
0  2024   11   22     0  BSB 500          0.0       0.0       0.0     0.0   
1  2024   11   22     0  BSB 501          0.0       0.0       0.0     0.0   
2  2024   11   22     0  BSB 502          0.0       0.0       0.0     0.0   
3  2024   11   22     0  BSB 503          0.0       0.0       0.0     0.0   
4  2024   11   22     0  BSB 504          0.0       0.0       0.0     0.0   

   Tamb_C  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  


In [44]:
# 9. AGREGACIÓN DIARIA DE DATOS REALES
def agregar_por_dia_reales(df_agrupado_hora):
    """
    Agrega datos reales por día (suma para energía y POA/GHI, promedio para temperaturas)
    Equivalente a la segunda agregación en Power Query
    """
    df_agrupado_dia = df_agrupado_hora.groupby(['ano', 'mes', 'dia', 'Planta']).agg({
        'Energia_kWh': 'sum',         # Suma de energía
        'POA_W/m2': 'sum',           # Suma para convertir a Wh/m2
        'GHI_W/m2': 'sum',           # Suma para convertir a Wh/m2  
        'Tmod_C': 'mean',            # Promedio de temperatura
        'Tamb_C': 'mean'             # Promedio de temperatura
    }).reset_index()
    
    # Renombrar columnas para claridad (como en Power Query)
    df_agrupado_dia = df_agrupado_dia.rename(columns={
        'Energia_kWh': 'Ener_kWh_Real',
        'POA_W/m2': 'POA_Wh/m2_Real',
        'GHI_W/m2': 'GHI_Wh/m2_Real'
    })
    
    print(f"Datos reales agrupados por día: {len(df_agrupado_dia):,} registros")
    return df_agrupado_dia

# Agregar datos reales por día
df_reales_dia = agregar_por_dia_reales(df_agrupado_hora)
print("Primeros registros de datos reales por día:")

Datos reales agrupados por día: 1,645 registros
Primeros registros de datos reales por día:
    ano  mes  dia   Planta  Ener_kWh_Real  POA_Wh/m2_Real  GHI_Wh/m2_Real  \
0  2024   11   22  BSB 500            0.0        0.000000          0.0000   
1  2024   11   22  BSB 501            0.0        0.000000          0.0000   
2  2024   11   22  BSB 502            0.0      221.788573        206.4937   
3  2024   11   22  BSB 503            0.0        0.000000          0.0000   
4  2024   11   22  BSB 504            0.0        0.000000          0.0000   

     Tmod_C    Tamb_C  
0  0.000000  0.000000  
1  0.000000  0.000000  
2  9.642877  4.774155  
3  0.000000  0.000000  
4  0.000000  0.000000  


In [45]:
df_reales_dia.head()

,ano,mes,dia,Planta,Ener_kWh_Real,POA_Wh/m2_Real,GHI_Wh/m2_Real,Tmod_C,Tamb_C
0,2024,11,22,BSB 500,0.0,0.000000,0.0000,0.000000,0.000000
1,2024,11,22,BSB 501,0.0,0.000000,0.0000,0.000000,0.000000
2,2024,11,22,BSB 502,0.0,221.788573,206.4937,9.642877,4.774155
3,2024,11,22,BSB 503,0.0,0.000000,0.0000,0.000000,0.000000
4,2024,11,22,BSB 504,0.0,0.000000,0.0000,0.000000,0.000000


In [46]:
# 10. COMBINACIÓN DE DATOS REALES Y MODELO
def combinar_reales_modelo(df_reales_dia, df_modelo_dia):
    """
    Combina datos reales y modelo por día
    Equivalente al join final en Power Query
    """
    if df_modelo_dia.empty:
        print("Advertencia: No hay datos del modelo para combinar")
        return df_reales_dia
    
    # Left Join entre datos reales y modelo
    df_combinado = df_reales_dia.merge(
        df_modelo_dia,
        left_on=['ano', 'mes', 'dia', 'Planta'],
        right_on=['Año', 'Mes', 'Día', 'Planta'],
        how='left',
        suffixes=('', '_modelo')
    )
    
    # Renombrar columnas del modelo (como en Power Query)
    df_combinado = df_combinado.rename(columns={
        'Ener_kWh': 'Ener_kWh_Modelo',
        'POA_Wh/m2': 'POA_Wh/m2_Modelo', 
        'GHI_Wh/m2': 'GHI_Wh/m2_Modelo'
    })
    
    # Eliminar columnas duplicadas
    columnas_a_eliminar = ['Año', 'Mes', 'Día']
    df_combinado = df_combinado.drop(columns=[col for col in columnas_a_eliminar if col in df_combinado.columns])
    
    print(f"Datos combinados reales y modelo: {len(df_combinado):,} registros")
    return df_combinado

# Combinar datos reales y modelo
df_final_combinado = combinar_reales_modelo(df_reales_dia, df_modelo_dia)
print("Primeros registros de datos combinados:")
print(df_final_combinado.head())
print("\nColumnas disponibles:")
print(df_final_combinado.columns.tolist())

Datos combinados reales y modelo: 1,645 registros
Primeros registros de datos combinados:
    ano  mes  dia   Planta  Ener_kWh_Real  POA_Wh/m2_Real  GHI_Wh/m2_Real  \
0  2024   11   22  BSB 500            0.0        0.000000          0.0000   
1  2024   11   22  BSB 501            0.0        0.000000          0.0000   
2  2024   11   22  BSB 502            0.0      221.788573        206.4937   
3  2024   11   22  BSB 503            0.0        0.000000          0.0000   
4  2024   11   22  BSB 504            0.0        0.000000          0.0000   

     Tmod_C    Tamb_C  Ener_kWh_Modelo  POA_Wh/m2_Modelo  GHI_Wh/m2_Modelo  
0  0.000000  0.000000              NaN               NaN               NaN  
1  0.000000  0.000000              NaN               NaN               NaN  
2  9.642877  4.774155              NaN               NaN               NaN  
3  0.000000  0.000000              NaN               NaN               NaN  
4  0.000000  0.000000              NaN               NaN      

In [47]:
df_final_combinado.head()

,ano,mes,dia,Planta,Ener_kWh_Real,POA_Wh/m2_Real,GHI_Wh/m2_Real,Tmod_C,Tamb_C,Ener_kWh_Modelo,POA_Wh/m2_Modelo,GHI_Wh/m2_Modelo
0,2024,11,22,BSB 500,0.0,0.000000,0.0000,0.000000,0.000000,NaN,NaN,NaN
1,2024,11,22,BSB 501,0.0,0.000000,0.0000,0.000000,0.000000,NaN,NaN,NaN
2,2024,11,22,BSB 502,0.0,221.788573,206.4937,9.642877,4.774155,NaN,NaN,NaN
3,2024,11,22,BSB 503,0.0,0.000000,0.0000,0.000000,0.000000,NaN,NaN,NaN
4,2024,11,22,BSB 504,0.0,0.000000,0.0000,0.000000,0.000000,NaN,NaN,NaN


In [16]:
# 11. TRANSPOSICIÓN FINAL - DATOS REALES VS MODELO POR MÉTRICA
def crear_tabla_transpuesta(df_final_combinado):
    """
    Crea la tabla transpuesta final con datos reales vs modelo por métrica
    """
    if df_final_combinado.empty:
        print("Error: No hay datos para transponer")
        return pd.DataFrame()
    
    # Preparar datos para transposición
    df_transponer = df_final_combinado.copy()
    
    # Crear identificador único por registro
    df_transponer['ID'] = df_transponer['ano'].astype(str) + "-" + \
                         df_transponer['mes'].astype(str).str.zfill(2) + "-" + \
                         df_transponer['dia'].astype(str).str.zfill(2) + "_" + \
                         df_transponer['Planta']
    
    # Definir métricas a transponer
    metricas = ['Ener_kWh', 'POA_Wh/m2', 'GHI_Wh/m2']
    
    df_transpuesto_list = []
    
    for metrica in metricas:
        col_real = f"{metrica}_Real"
        col_modelo = f"{metrica}_Modelo"
        
        if col_real in df_transponer.columns:
            # Datos reales
            df_real = df_transponer[['ano', 'mes', 'dia', 'Planta', 'ID', col_real]].copy()
            df_real['Tipo'] = 'Real'
            df_real['Metrica'] = metrica
            df_real['Valor'] = df_real[col_real]
            df_real = df_real[['ano', 'mes', 'dia', 'Planta', 'ID', 'Tipo', 'Metrica', 'Valor']]
            df_transpuesto_list.append(df_real)
        
        if col_modelo in df_transponer.columns:
            # Datos modelo
            df_modelo = df_transponer[['ano', 'mes', 'dia', 'Planta', 'ID', col_modelo]].copy()
            df_modelo['Tipo'] = 'Modelo'
            df_modelo['Metrica'] = metrica
            df_modelo['Valor'] = df_modelo[col_modelo]
            df_modelo = df_modelo[['ano', 'mes', 'dia', 'Planta', 'ID', 'Tipo', 'Metrica', 'Valor']]
            df_transpuesto_list.append(df_modelo)
    
    if df_transpuesto_list:
        df_transpuesto = pd.concat(df_transpuesto_list, ignore_index=True)
        
        # Crear tabla final pivoteada
        df_pivot = df_transpuesto.pivot_table(
            index=['ano', 'mes', 'dia', 'Planta', 'Metrica'],
            columns='Tipo',
            values='Valor',
            aggfunc='first'
        ).reset_index()
        
        # Aplanar nombres de columnas
        df_pivot.columns.name = None
        
        # Crear fecha
        df_pivot['Fecha'] = pd.to_datetime(
            df_pivot['ano'].astype(str) + '-' + 
            df_pivot['mes'].astype(str) + '-' + 
            df_pivot['dia'].astype(str)
        )
        
        # Reordenar columnas
        columnas_finales = ['Fecha', 'ano', 'mes', 'dia', 'Planta', 'Metrica', 'Real', 'Modelo']
        df_pivot = df_pivot[[col for col in columnas_finales if col in df_pivot.columns]]
        
        # Ordenar por fecha y planta
        df_pivot = df_pivot.sort_values(['Fecha', 'Planta', 'Metrica']).reset_index(drop=True)
        
        print(f"Tabla transpuesta creada: {len(df_pivot):,} registros")
        print(f"Métricas disponibles: {df_pivot['Metrica'].unique()}")
        print(f"Plantas: {df_pivot['Planta'].unique()}")
        
        return df_pivot
    else:
        print("Error: No se pudieron crear datos transpuestos")
        return pd.DataFrame()

# Crear tabla transpuesta final
df_resultado_final = crear_tabla_transpuesta(df_final_combinado)

if not df_resultado_final.empty:
    print("\n=== RESULTADO FINAL ===")
    print(f"Registros totales: {len(df_resultado_final):,}")
    print(f"Rango de fechas: {df_resultado_final['Fecha'].min()} a {df_resultado_final['Fecha'].max()}")
    print("\nPrimeros registros:")
    print(df_resultado_final.head(10))
    
    print("\nResumen por métrica:")
    print(df_resultado_final.groupby(['Metrica']).agg({
        'Real': ['count', 'mean', 'sum'],
        'Modelo': ['count', 'mean', 'sum']
    }).round(2))
else:
    print("Error: No se pudo generar la tabla final")

Tabla transpuesta creada: 4,935 registros
Métricas disponibles: ['Ener_kWh' 'GHI_Wh/m2' 'POA_Wh/m2']
Plantas: ['BSB 500' 'BSB 501' 'BSB 502' 'BSB 503' 'BSB 504']

=== RESULTADO FINAL ===
Registros totales: 4,935
Rango de fechas: 2024-11-22 00:00:00 a 2025-10-16 00:00:00

Primeros registros:
       Fecha   ano  mes  dia   Planta    Metrica         Real  Modelo
0 2024-11-22  2024   11   22  BSB 500   Ener_kWh  1450.998207     NaN
1 2024-11-22  2024   11   22  BSB 500  GHI_Wh/m2  1304.022213     NaN
2 2024-11-22  2024   11   22  BSB 500  POA_Wh/m2  1888.481851     NaN
3 2024-11-22  2024   11   22  BSB 501   Ener_kWh  1495.700171     NaN
4 2024-11-22  2024   11   22  BSB 501  GHI_Wh/m2  2437.168010     NaN
5 2024-11-22  2024   11   22  BSB 501  POA_Wh/m2  2235.809590     NaN
6 2024-11-22  2024   11   22  BSB 502   Ener_kWh  1815.881284     NaN
7 2024-11-22  2024   11   22  BSB 502  GHI_Wh/m2  1851.414701     NaN
8 2024-11-22  2024   11   22  BSB 502  POA_Wh/m2  1905.712254     NaN
9 2024-1

In [17]:
# 12. EXPORTAR RESULTADOS
def exportar_resultados(df_resultado_final, ruta_base):
    """
    Exporta los resultados a archivos CSV y Excel
    """
    if df_resultado_final.empty:
        print("No hay datos para exportar")
        return
    
    # Crear carpeta de salida
    ruta_salida = os.path.join(ruta_base, "05 OUT", "PROCESAMIENTO_DATOS")
    os.makedirs(ruta_salida, exist_ok=True)
    
    # Nombre del archivo con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_base = f"modelo_y_real_transpuesto_{timestamp}"
    
    # Exportar a CSV
    archivo_csv = os.path.join(ruta_salida, f"{nombre_base}.csv")
    df_resultado_final.to_csv(archivo_csv, index=False, encoding='utf-8-sig')
    print(f"Archivo CSV exportado: {archivo_csv}")
    
    # Exportar a Excel con múltiples hojas
    archivo_excel = os.path.join(ruta_salida, f"{nombre_base}.xlsx")
    with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
        # Hoja principal con todos los datos
        df_resultado_final.to_excel(writer, sheet_name='Datos_Transpuestos', index=False)
        
        # Hoja por métrica
        for metrica in df_resultado_final['Metrica'].unique():
            df_metrica = df_resultado_final[df_resultado_final['Metrica'] == metrica].copy()
            nombre_hoja = metrica.replace('/', '_').replace(' ', '_')[:30]  # Límite de Excel
            df_metrica.to_excel(writer, sheet_name=nombre_hoja, index=False)
        
        # Hoja de resumen
        resumen = df_resultado_final.groupby(['Planta', 'Metrica']).agg({
            'Real': ['count', 'mean', 'sum'],
            'Modelo': ['count', 'mean', 'sum']
        }).round(2)
        resumen.to_excel(writer, sheet_name='Resumen')
    
    print(f"Archivo Excel exportado: {archivo_excel}")
    
    # Crear tabla resumen para visualización
    print("\n=== RESUMEN DE EXPORTACIÓN ===")
    print(f"Total de registros exportados: {len(df_resultado_final):,}")
    print(f"Plantas: {', '.join(df_resultado_final['Planta'].unique())}")
    print(f"Métricas: {', '.join(df_resultado_final['Metrica'].unique())}")
    print(f"Período: {df_resultado_final['Fecha'].min()} a {df_resultado_final['Fecha'].max()}")

# Exportar resultados
exportar_resultados(df_resultado_final, ruta_base)

# Mostrar muestra final por planta y métrica
if not df_resultado_final.empty:
    print("\n=== MUESTRA FINAL POR PLANTA Y MÉTRICA ===")
    for planta in df_resultado_final['Planta'].unique()[:2]:  # Mostrar solo 2 plantas
        print(f"\n--- {planta} ---")
        df_planta = df_resultado_final[df_resultado_final['Planta'] == planta]
        for metrica in df_planta['Metrica'].unique():
            df_muestra = df_planta[df_planta['Metrica'] == metrica].head(3)
            print(f"\n{metrica}:")
            print(df_muestra[['Fecha', 'Real', 'Modelo']].to_string(index=False))

Archivo CSV exportado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 OUT\PROCESAMIENTO_DATOS\modelo_y_real_transpuesto_20251104_103101.csv
Archivo Excel exportado: C:\Users\jhon.galeano.m\OneDrive - ApplusGlobal\Archivos de LAURA CAROLINA ESQUIVEL - ISAGEN- Bosques Solares\05 OUT\PROCESAMIENTO_DATOS\modelo_y_real_transpuesto_20251104_103101.xlsx

=== RESUMEN DE EXPORTACIÓN ===
Total de registros exportados: 4,935
Plantas: BSB 500, BSB 501, BSB 502, BSB 503, BSB 504
Métricas: Ener_kWh, GHI_Wh/m2, POA_Wh/m2
Período: 2024-11-22 00:00:00 a 2025-10-16 00:00:00

=== MUESTRA FINAL POR PLANTA Y MÉTRICA ===

--- BSB 500 ---

Ener_kWh:
     Fecha        Real  Modelo
2024-11-22 1450.998207     NaN
2024-11-23 1395.172037     NaN
2024-11-24 1741.048064     NaN

GHI_Wh/m2:
     Fecha        Real  Modelo
2024-11-22 1304.022213     NaN
2024-11-23 1237.836428     NaN
2024-11-24 1871.946676     NaN

POA_Wh/m2:
     Fecha        Real  Mod